# 加速：多線程爬蟲



* 了解知乎 API 使用方式與回傳內容
* 撰寫程式存取 API 且添加標頭

## 作業目標

* 找一個之前實作過的爬蟲改用多線程改寫，比較前後時間的差異。





In [1]:
#實作範例D023
import requests
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

In [2]:
ETtoday_url = "https://www.ettoday.net/news/focus/%E8%B2%A1%E7%B6%93/"  #財經新聞

In [3]:
browser = webdriver.Chrome()
browser.get(ETtoday_url)  # 打開瀏覽器並連到東森新聞雲網頁

SCROLL_PAUSE_TIME = 1


#
# 以下是用Selenium模擬下拉網頁動作，讓網頁更新
#
last_height = browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
while True:

    # Scroll down to the bottom.
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # Wait to load the page.
    time.sleep(SCROLL_PAUSE_TIME)

    # Calculate new scroll height and compare with last scroll height.
    new_height = browser.execute_script("return document.body.scrollHeight")

    if new_height == last_height:
        print("到達頁面底端")
        break

    last_height = new_height
    print("網頁更新中...")


網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
到達頁面底端
17.124372243881226


In [26]:
#
# 這裡先建立一個函數，其功能是連到外部連結，並爬取新聞內容。
#
def getNewsDetailContent(idx, title, link_url):
    result = "----------------------------------------------------------------------"
    result = result + ("\n[%d] %s\n" % (idx, title))
    resp = requests.get(link_url)
    resp.encoding = 'utf-8'


    soup = BeautifulSoup(resp.text, 'lxml')
    news_content = soup.find("div", attrs={'class':'story'}).find_all("p")
    for p in news_content:
        """
        .string屬性說明：
        (1) 若當前tag節點底下沒有其他tag子節點，會直接抓取內容(返回"NavigableString")
        (2) 若當前tag節點底下只有唯一的一個tag子節點，也會直接抓取tag子節點的內容(返回"NavigableString")
        (3) 但若當前tag節點底下還有很多個tag子節點，.string就無法判斷，(返回"None")
        """
        if ((p.string) is not None):
            result += p.string
    
    print(result)

In [27]:
# 爬取網頁內容，解析後萃取新聞摘要
html = browser.page_source
soup = BeautifulSoup(html, "lxml")
all_news = soup.find("div", attrs={'class':'block block_1 infinite_scroll'})

news_block = all_news.find_all('div', attrs={'class':'piece clearfix'})

startTime = time.time() # 開始計時
for i, news_item in enumerate(news_block):
    
    news_body = news_item.find('h3')
    #print("\n[%d] %s\n" % (i, news_body.a.string))
    
    # 
    # 連到外部連結，擷取詳細新聞內容
    #
    externalLink = "https://www.ettoday.net" + news_body.a["href"]
    getNewsDetailContent(i,news_body.a.string,externalLink)
finishTime = time.time() # 結束計時
print(finishTime - startTime) # 正常情況的爬蟲所需時間

----------------------------------------------------------------------
[0] 口罩核心原料飆漲10倍也不賣國外　經濟部掛保證：台灣產能優先
▲經濟部粉絲團貼文指出，在熔噴布的國際行情飆漲到原先的8-10倍之際，台灣有能力製造熔噴布的企業仍優先把產能提供給國內使用。（圖／翻攝自經濟部粉絲團）記者邱倢芯／台北報導新冠肺炎（COVID-19）疫情持續延燒，各國口罩供給仍緊，就連口罩原料也是各國搶購；根據經濟部臉書粉絲團指出，目前口罩核心原料「熔噴不織布」的國際行情已經飆漲到原先的8-10倍，但台灣有能力製造熔噴布的企業，都配合政府，優先把產能提供給國內使用。粉絲團貼文指出，前陣子總統蔡英文、經濟部長沈榮津，以及桃園市長鄭文燦，一起到位在桃園的敏成公司表達感謝，因為敏成是台灣生產口罩時，最大的熔噴布供應者。而在熔噴布的國際行情飆漲到原先的8-10倍之際，敏成仍優先把產能提供給國內使用，真正是「足感心」。另外，經濟部粉絲團的貼文中也指出，過去台灣的口罩日產量約188萬片，以往都是口罩的進口國，才短短一兩個月，現在已經突破1,300萬片日產量，連假後更要挑戰1,500萬片的水準。行有餘力，台灣還能以類似捐出一日所得的方式，用1,000萬片口罩，支持其他國家的醫護人員，為全世界的防疫工作貢獻心力，讓世界看到台灣製造的實力。
----------------------------------------------------------------------
[1] 美國首家銀行宣告倒閉！疫情下金融業第1槍　FDIC將接管
▲疫情衝擊，美國出現首家倒閉的銀行。（圖／路透）圖文／CTWANT美國新冠肺炎疫情嚴峻之際，西維吉尼亞州的Barboursville的小型銀行The First State Bank宣布破產，成為疫情後首家倒下的銀行。美國聯邦存款保險公司（FDIC）表示，原本的業務將由MVB Bank接管，客戶將會自動轉到MVB Bank旗下。FDIC指出，The First State Bank長期以來都有資本和資產質量問題，自2015年以來財務一直陷入困境，直到銀行2019年12月31日的財務報告顯示，資本水平太低，根據聯邦和州法律已無法繼續經營。為保障存款人，FDIC表示，The First S

In [28]:
import _thread

# 爬取網頁內容，解析後萃取新聞摘要
html = browser.page_source
soup = BeautifulSoup(html, "lxml")
all_news = soup.find("div", attrs={'class':'block block_1 infinite_scroll'})

news_block = all_news.find_all('div', attrs={'class':'piece clearfix'})

startTime = time.time() # 開始計時
for i, news_item in enumerate(news_block):
    #print("----------------------------------------------------------------------")
    news_body = news_item.find('h3')
    #print("\n[%d] %s\n" % (i, news_body.a.string))
    
    # 
    # 連到外部連結，擷取詳細新聞內容
    #
    externalLink = "https://www.ettoday.net" + news_body.a["href"]
    _thread.start_new_thread(getNewsDetailContent, (i,news_body.a.string,externalLink,))
finishTime = time.time() # 結束計時
print(finishTime - startTime) # 正常情況的爬蟲所需時間

0.06882262229919434
----------------------------------------------------------------------
[7] 轉黑！3月出口金額估年減上看5%　專家：傷害延續到第2季
▲疫情持續發燒，3月出口金額不樂觀（圖／記者林世文攝）記者吳靜君／台北報導財政部週三（8日）將公布進出口統計，根據財政部預估，受到新冠肺炎的影響，國際多國已經採行封城、隔離等政策，實質傷害到實體經濟，3月的出口值將為負成長，較去年同期減少2%到5%，甚至專家預估，將年減達到5.6%左右，加上中國大陸3月仍有停工情況，將會衝擊塑化及機械的出口動能。根據歷史資料顯示，去（2019）年3月出口286.8億美元，而財政部估計，3月出口金額恐怕無法到達去年的成績，可能減少2%到5%之間。中央大學經濟系教授徐之強表示，現在新冠肺炎（COVID-19）的疫情在美國等地方持續擴散，而我國對美國的貿易依存度同樣相當高，因此預估疫情衝擊出口情形將延續到第2季。財政部官員表示，中美貿易戰的正向發展、5G以及AI的新興商機、台商產能移回台灣等正項因素仍在；但新冠肺炎疫情仍肆虐全球，形成短期衝擊，後續發展也尚未明朗，全球的消費和原物料的價格已經受到影響，確實會壓縮我國出口動能。以2月的出口值來說，原本財政部是預估可以年成長三成左右，但是僅成長24.9%，出口值253.8億美元，顯示新冠肺炎確實衝擊到我國的出口金額，單月出口值衝擊約15到18億美元。元大寶華綜合經濟研究院高級研究員賴怡欣則指出，疫情導致全球前所未有的封城、隔離等措施，這些都不利全球產業鏈的生產與運輸，各國的需求一度跌破20美元，原物料價格也明顯走跌。經濟部所公布的2月外銷訂單來看，在工作日還比較多的情形下，仍比去年同期少了0.8%，顯見出口仍相當不利。以經濟部2月外銷訂單指數而言，賴怡欣認為，即使今年2月的工作天數足足比去年多了5天，但2月外銷訂單金額仍年減0.8％，足見疫情衝擊甚深。雖然國際經濟進入寒冬，但台灣因疫情相對控制得宜，可能會有部分轉單效應，預估3月出口值應為負成長5.6%。徐之強則認為，由於中國大陸、美國、歐洲等各國都有封城、隔離等政策，生產鏈恐怕產生斷鏈危機，第1季經濟表現受到衝擊，加上各行業尤其是航空業等幾乎已經沒有運行等，出口低迷恐怕會延續到第2季。
------